# I - Setup
This notebook will set up everything for the tutorial. This notebook assumes that nothing has been set up previously and will create everything from scratch. The necessary steps are broken up into the following sections:

**Note:** By using these notebooks on GPU VMs, you agree to the [NVIDIA Software License](http://www.nvidia.com/content/DriverDownload-March2009/licence.php?lang=us).

* [Install tools and dependencies](#section1)
* [Azure account login](#section2)
* [Setup](#section3)
* [Create Azure resources](#section4)
* [Batch Shipyard Configuration](#section5)
* [Create Azure Batch Pool](#section6)
* [View Resources on Azure Portal or Batch Labs](#section7)

<a id='section1'></a>

## Install tools and dependencies
Install Batch Shipyard and Azure CLI 2.0

In [ ]:
!git clone https://github.com/Azure/batch-shipyard.git

Normally you would use `install.sh` or `install.cmd` helper scripts to install, but due to the Notebook environment, we will instead install with the `requirements.txt` file directly.

In [ ]:
!pip install -r batch-shipyard/requirements.txt

Azure CLI 2.0 will also be installed to help us in provisioning Azure Batch and Storage accounts.

In [ ]:
!pip install -I azure-cli

We'll create an alias for invoking Batch Shipyard which points to a `config` directory which will hold our json config files (this directory will be created at a later step).

In [ ]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working:

In [ ]:
shipyard --version

<a id='section2'></a>

## Azure account login
The command below will initiate a login to your Azure account. You will see a url to browse to where you will enter the specified code. This will log you into the Azure account within the Azure CLI.

In [ ]:
!az login -o table

If you have multiple subscriptions you can select the one you need with the command below. This will not be necessary for your assigned Azure Pass account for the workshop.

In [ ]:
selected_subscription = '"YOUR TEAM SUBSCRIPTION"' # Replace with the name of your subscription
!az account set --subscription $selected_subscription

**Note:** If you cannot login with the Azure CLI, you can create Azure Batch and Storage accounts on the [Azure Portal](https://portal.azure.com).
- [Instructions for creating an Azure Batch Account](https://docs.microsoft.com/en-us/azure/batch/batch-account-create-portal#batch-service-mode)
- [Instructions for creating an Azure Storage Account](https://docs.microsoft.com/en-us/azure/storage/storage-create-storage-account#create-a-storage-account) (You can create an "Auto Storage" account at the same time as your Batch Account on the portal instead)

Please pay attention to special instructions regarding Azure Portal created accounts below.

<a id='section3'></a>

## Setup

First we will need to register the Azure Batch service as a resource provider for the Azure subscription. The following will do so and poll until the registration process has completed. This will take approximately 30 seconds to complete.

**Note:** This registration process needs to be performed only once for the Azure subscription. If you created your Azure Batch account via the Azure Portal, you do not need to perform this action as it is done automatically for you.

In [ ]:
import time

# register resource provider with subscription
print('Registering Microsoft.Batch with subscription. Please be patient for this process.')
!az provider register -n Microsoft.Batch

# poll until registration completed
while True:
    status = !az provider show -n Microsoft.Batch -o table
    if status[-1].split()[-1] == 'Registered':
        print('\n'.join(status))
        break
    time.sleep(2)

Now we will define the various names for the resources needed to run Azure Batch jobs.

**Note:** If you manually created your accounts on the Azure Portal, you will need to modify `GROUP_NAME`, `BATCH_ACCOUNT_NAME` and `STORAGE_ACCOUNT_NAME` accordingly.

In [ ]:
import json
import os
import uuid
import random

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

LOCATION = 'eastus' # We are setting everything up in East US
                    # Be aware that you need to set things up in a region that has GPU VMs (N-Series)

# load base CNTK image from Docker hub
IMAGE_NAME = "microsoft/cntk:2.0-gpu-python3.5-cuda8.0-cudnn5.1"

# Please do not modify below unless you created your accounts on the Azure Portal
short_uuid = str(uuid.uuid4())[:8]

GROUP_NAME = "batch{uuid}rg".format(uuid=short_uuid)
BATCH_ACCOUNT_NAME = "batch{uuid}ba".format(uuid=short_uuid)
STORAGE_ACCOUNT_NAME = "batch{uuid}st".format(uuid=short_uuid)
STORAGE_ALIAS = "mystorageaccount"
STORAGE_ENDPOINT = "core.windows.net"

<a id='section4'></a>

## Create Azure resources

### Create Resource Group
Azure encourages the use of resource groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the Resource Group.

In [ ]:
!az group create -n $GROUP_NAME -l $LOCATION -o table

### Create Batch and Storage accounts
Here we simply crate the Batch and Storage accounts. Once we have created the accounts we can the use the Azure CLI to query them and obtain the **batch_account_key**, **batch_service_url** and **storage_account_key** which we will need for our Batch Shipyard configuration files later.

In [ ]:
json_data = !az storage account create -l $LOCATION -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME --sku Standard_LRS
print('Storage account {} provisioning state: {}'.format(STORAGE_ACCOUNT_NAME, json.loads(''.join(json_data))['provisioningState']))

In [ ]:
json_data = !az batch account create -l $LOCATION -n $BATCH_ACCOUNT_NAME -g $GROUP_NAME --storage-account $STORAGE_ACCOUNT_NAME
print('Batch account {} provisioning state: {}'.format(BATCH_ACCOUNT_NAME, json.loads(''.join(json_data))['provisioningState']))

Next we retrieve the **batch_account_key**, **batch_service_url** and **storage_account_key** which we will need for the Batch Shipyard configuration files further down.

**Note:** If you created your Batch and Storage accounts in the Azure Portal, you will need to retrieve your keys in the Portal. Then set `batch_account_key`, `batch_service_url`, and `storage_account_key` to their appropriate values instead of the Azure CLI callouts.

In [ ]:
json_data = !az batch account keys list -n $BATCH_ACCOUNT_NAME -g $GROUP_NAME
batch_account_key = json.loads(''.join(json_data))['primary']

In [ ]:
json_data = !az batch account list -g $GROUP_NAME
batch_service_url = 'https://'+json.loads(''.join(json_data))[0]['accountEndpoint']

In [ ]:
json_data = !az storage account keys list -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME
storage_account_key = json.loads(''.join(json_data))[0]['value']

Save credentials required for other notebooks.

In [ ]:
account_information = {
    "IMAGE_NAME": IMAGE_NAME,
    "LOCATION": LOCATION,
    "resource_group": GROUP_NAME,
    "STORAGE_ALIAS": STORAGE_ALIAS,
    "storage_account_key": storage_account_key,
    "storage_account_name": STORAGE_ACCOUNT_NAME,
}
write_json_to_file(account_information, 'account_information.json')

<a id='section5'></a>

## Batch Shiyard configuration
In order to execute a job on Batch Shipyard you need a minimum of four configuration files. We will set three of them here and leave the job one for later.
* [credentials](#credentials)
* [configuration](#configuration)
* [pool](#pool)


<a id='credentials'></a>

### Credentials
Here we define all the credentials necessary for Batch Shipyard to connect to Azure for resource provisioning and executing our jobs.

In [ ]:
credentials = {
    "credentials": {
        "batch": {
            "account_key": batch_account_key,
            "account_service_url": batch_service_url
        },
        "storage": {
            STORAGE_ALIAS : {
                    "account": STORAGE_ACCOUNT_NAME,
                    "account_key": storage_account_key,
                    "endpoint": STORAGE_ENDPOINT
            }
        }
    }
}

<a id='configuration'></a>

### Configuration
The config mainly contains the configuration for Batch Shipyard. Here we simply define the storage alias that Batch Shipyard should use as well as the Docker image to use.

In [ ]:
config = {
    "batch_shipyard": {
        "storage_account_settings": STORAGE_ALIAS
    },
    "global_resources": {
        "docker_images": [
            IMAGE_NAME
        ]
    }
}

<a id='pool'></a>

### Pool
This is where we define the properties of the compute pool we wish to create. The configuration below creates a pool that is made up of a three NC6 VM running Ubuntu 16.04. If you wish to run a job that uses GPU accelerated compute, as we will be doing for these notebooks, then you will need to choose a VM from the NC series. Here we will allocate 3 `STANDARD_NC6` instances.

Under resource files we are pulling in two scripts that will be run as part of the pool creation. They simply download and prepare the CIFAR10 data. These files can be found in the github repository.

In [ ]:
POOL_ID = 'gpupool'

pool = {
    "pool_specification": {
        "id": POOL_ID,
        "vm_configuration": {
            "platform_image": {
                "publisher": "Canonical",
                "offer": "UbuntuServer",
                "sku": "16.04-LTS"
            },
        },
        "vm_size": "STANDARD_NC6",
        "vm_count": {
            "dedicated": 3
        },
        "ssh": {
            "username": "docker"
        },
        "reboot_on_start_task_failed": False,
        "block_until_all_global_resources_loaded": True,
        "resource_files": [
            {
                "file_path": "cifar_data_processing.py",
                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/cifar_data_processing.py",
                "file_mode":'0777'
            },
            {
                "file_path": "convert_cifar10.sh",
                "blob_source": "https://batchshipyardexamples.blob.core.windows.net/code/convert_cifar10.sh",
                "file_mode":'0777'
            }
        ],
         "additional_node_prep_commands": [
            "/bin/bash convert_cifar10.sh {} $AZ_BATCH_NODE_SHARED_DIR/data".format(IMAGE_NAME)
        ]
    }
}

In [ ]:
!mkdir -p config # Create config file directory where we will store all our Batch Shipyard configuration files

In [ ]:
write_json_to_file(credentials, os.path.join('config', 'credentials.json'))

In [ ]:
write_json_to_file(config, os.path.join('config', 'config.json'))

In [ ]:
write_json_to_file(pool, os.path.join('config', 'pool.json'))

Please run the following cell and copy/paste the information output from the cell into a scratch space (e.g., Notepad), just in case for recovery purposes. Note that this will output credential secrets for your accounts, so do not openly distribute:

In [ ]:
print('IMAGE_NAME = "{}"'.format(IMAGE_NAME))
print('GROUP_NAME = "{}"'.format(GROUP_NAME))
print('LOCATION = "{}"'.format(LOCATION))
print('BATCH_ACCOUNT_NAME = "{}"'.format(BATCH_ACCOUNT_NAME))
print('batch_account_key = "{}"'.format(batch_account_key))
print('batch_service_url = "{}"'.format(batch_service_url))
print('STORAGE_ACCOUNT_NAME = "{}"'.format(STORAGE_ACCOUNT_NAME))
print('STORAGE_ALIAS = "{}"'.format(STORAGE_ALIAS))
print('storage_account_key = "{}"'.format(storage_account_key))

<a id='section6'></a>

## Create Azure Batch Pool
Before we do anything we need to create the pool for Batch Shipyard jobs to run on. This can take a little bit of time so please be patient while the compute nodes are allocated from the Azure Cloud and the Docker images are pre-loaded on to the compute nodes.

**Note:** As soon as one node enters `ComputeNodeState.idle` then you can proceed to the next notebook. You do not need to wait for all nodes to enter this state to begin the next notebook.

In [ ]:
shipyard pool add -y

Once the pool is created we can confirm everything by running the command below.

In [ ]:
shipyard pool listnodes

<a id='section7'></a>
## View Resources on Azure Portal or Batch Labs

If you want a graphical interface for your Azure Batch resources, you can view them on the [Azure Portal](https://portal.azure.com) or with [Batch Labs](https://github.com/Azure/BatchLabs).

[Next notebook: Single GPU Training](02_Single_GPU_Training.ipynb)